# Giriş

29/05/2021 - [@mebaysan](https://github.com/mebaysan)

Bu notebook'da [Plotly](https://plotly.com/python/) 'de [subplots](https://plotly.com/python/subplots/) kavramını öğrenmeye çalışacağız.

In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Subplots Nedir?

Bir grid içerisinde düzenlenmiş mini plotlar diyebiliriz.
- Bir veri setine ait farklı plotları
- Bir veri setine ait farklı alt kümelere ait aynı plotları vb. göstermek için kullanabiliriz

## Subplots Kullanımı
- `from plotly.subplots import make_subplots` ile fonksiyonu çağırıyoruz
- *Unutmayalım ki bu dökümanın hazırlandığı günlerde plotly express ile subplot kullanımı desteklenmemektedir.*

# Veri Setini Yükleyelim & İnceleyelim

Bu notebook'umuzda muhtemelen hepimizin bildiği seaborn içerisindeki `tips` veri setini kullanacağız

In [4]:
df = sns.load_dataset('tips')

In [5]:
df.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244 entries, 0 to 243
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   total_bill  244 non-null    float64 
 1   tip         244 non-null    float64 
 2   sex         244 non-null    category
 3   smoker      244 non-null    category
 4   day         244 non-null    category
 5   time        244 non-null    category
 6   size        244 non-null    int64   
dtypes: category(4), float64(2), int64(1)
memory usage: 7.3 KB


In [10]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
total_bill,244.0,19.785943,8.902412,3.07,13.3475,17.795,24.1275,50.81
tip,244.0,2.998279,1.383638,1.00,2.0000,2.900,3.5625,10.00
size,244.0,2.569672,0.951100,1.00,2.0000,2.000,3.0000,6.00


# İlk Subplotu Oluşturalım (1 X 2)

In [31]:
# bar plot oluşturacağımız için veriyi grupluyoruz
grouped_df = df.groupby(['sex', 'day'], as_index=False).sum()
grouped_df.head()

,sex,day,total_bill,tip,size
0,Male,Thur,561.44,89.41,73
1,Male,Fri,198.57,26.93,21
2,Male,Sat,1227.35,181.95,156
3,Male,Sun,1269.46,186.78,163
4,Female,Thur,534.89,82.42,79


In [50]:
fig = make_subplots(rows=1,  # oluşturacağımız grid (zemin) kaç satır olsun
                    cols=2  # oluşturacağımız grid (zemin) kaç sütun olsun
                    )

In [51]:
fig.add_trace(go.Bar(x=grouped_df[grouped_df['sex'] == 'Male']['day'],
                     y=grouped_df[grouped_df['sex'] == 'Male']['tip'],
                     name='Male'  # bu grafiğin lejanttaki adı
                     ),
              row=1,  # eklediğimiz grafik hangi satıra
              col=1  # hangi sütuna gelsin
              )
fig.add_trace(go.Bar(x=grouped_df[grouped_df['sex'] == 'Female']['day'],
                     y=grouped_df[grouped_df['sex'] == 'Female']['tip'], name='Female'), row=1, col=2)

## Subplotlarımızı Güzelleştirelim

### Plotlara Title Verelim

In [52]:
grouped_df = df.groupby(['sex', 'day'], as_index=False).sum()

In [53]:
fig = make_subplots(rows=1,
                    cols=2,
                    subplot_titles=[  # sırasıyla subplot başlıklarını giriyoruz
                        'Erkekler',  # 1. subplot başlığı
                        'Kadınlar'  # 2. subplot başlığı
                    ]
                    )

In [54]:
fig.add_trace(go.Bar(x=grouped_df[grouped_df['sex'] == 'Male']['day'],
                     y=grouped_df[grouped_df['sex'] == 'Male']['tip'], name='Male'),
              row=1,
              col=1
              )
fig.add_trace(go.Bar(x=grouped_df[grouped_df['sex'] == 'Female']['day'],
                     y=grouped_df[grouped_df['sex'] == 'Female']['tip'], name='Female'), row=1, col=2)

fig.update_layout(title='Günlere Göre Toplam Bahşişler',  # ana grid başlığı
                  title_x=0.5  # ana grid başlık pozisyonu
                  )

### Plotlar arasında X Eksenini Paylaştıralım

`shared_xaxes=True` parametresi ile x eksenini,

aynı şekilde `shared_yaxes=True` parametresi ile y eksenini paylaştırabiliriz.

In [66]:
grouped_df = df.groupby(['sex', 'day'], as_index=False).sum()

In [67]:
fig = make_subplots(rows=len(grouped_df['sex'].unique()),  # unique değer sayısı kadar satır olacak
                    cols=1,
                    shared_xaxes=True  # plotlar x exsenini paylaşacak
                    )

In [68]:
row = 1
for trace in grouped_df['sex'].unique():
    fig.add_trace(go.Bar(x=grouped_df[grouped_df['sex'] == trace]['day'],
                         y=grouped_df[grouped_df['sex'] == trace]['tip'], name=trace),
                  row=row,
                  col=1
                  )
    row += 1

In [69]:
fig

# Plot Katmanlama (Layering)

- Layering plots = çoklu plotların birbiri üstüne gelmesi ile oluşur
- Ayrılmış grid yoktur
- `add_trace` fonksiyonu kullanılır
- Bir figür içerisine birden fazla plot çizdirmek diyebiliriz

In [81]:
grouped_df = df.groupby(['sex', 'day'], as_index=False).sum()

In [82]:
fig = go.Figure() # boş bir figür oluşturuyoruz

In [83]:
fig.add_trace(go.Bar(x=grouped_df[grouped_df['sex']=='Male']['day'],
                    y=grouped_df[grouped_df['sex']=='Male']['tip'],name='Male'))

In [84]:
fig.add_trace(go.Scatter(x=grouped_df[grouped_df['sex']=='Female']['day'],
                    y=grouped_df[grouped_df['sex']=='Female']['tip'],mode='lines+markers',name='Female'))